In [1]:
import torch
import torch.nn as nn
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

<div class="jumbotron">
    <h1 class="display-1">GPU与深度学习</h1>
    <hr class="my-4">
    <p>主讲：李岩</p>
    <p>管理学院</p>
    <p>liyan@cumtb.edu.cn</p>
</div>

- 自2000年以来，GPU性能每十年增长1000倍

- 如何使用单个GPU，然后是如何使用多个GPU和多个服务器（具有多个GPU）进行深度学习

## 显卡类型

### NVIDIA显卡

- 确保电脑至少安装了一个NVIDIA GPU
- 下载并安装NVIDIA驱动 [https://www.nvidia.cn/Download/index.aspx?lang=cn](https://www.nvidia.cn/Download/index.aspx?lang=cn)
- 下载并安装CUDA  [https://developer.nvidia.com/cuda-downloads](https://developer.nvidia.com/cuda-downloads)

- `Windows`系统查看`GPU`方法

<div class="row">
    <div class="col col-md-6">
        <center><img src="../img/5_deep_learning_computation/winGPUChk1.png" width=60%></center>
    </div>
    <div class="col col-md-6">
        <center><img src="../img/5_deep_learning_computation/winGPUChk2.png" width=80%></center>
    </div>
</div>

- `Mac`系统查看`GPU`方法

<div class="row">
    <div class="col col-md-4">
        <center><img src="../img/5_deep_learning_computation/macGPUChk1.png" width=70%></center>
    </div>
    <div class="col col-md-4">
        <center><img src="../img/5_deep_learning_computation/macGPUChk2.png" width=80%></center>
    </div>
    <div class="col col-md-4">
        <center><img src="../img/5_deep_learning_computation/macGPUChk3.png" width=100%></center>
    </div>
</div>

- 选择`PyTorch`版本

<center><img src="../img/5_deep_learning_computation/pytorchNVIDIA.png" width=100%></center>

In [1]:
# 使用`nvidia-smi`命令来查看显卡信息

!nvidia-smi

Wed Dec  7 16:36:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.106.00   Driver Version: 460.106.00   CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:1B.0 Off |                    0 |
| N/A   27C    P0    47W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  Off  | 00000000:00:1D.0 Off |                    0 |
| N/A   

### Mac的M1显卡

- 选择`PyTorch`版本

<center><img src="../img/5_deep_learning_computation/pytorchMac.png" width=100%></center>

### AMD显卡

- 需要开启Windows的Linux子系统
- 需要使用`Direct-ML`

1. 开启Windows的linux子系统(wsl)
    - 开启方法参照 [https://learn.microsoft.com/zh-cn/training/modules/wsl-introduction/install-and-setup](https://learn.microsoft.com/zh-cn/training/modules/wsl-introduction/install-and-setup)
1. 升级wsl内核
    - 找到微软发布的内核更新安装文件 msi [https://www.catalog.update.microsoft.com/Search.aspx?q=wsl](https://www.catalog.update.microsoft.com/Search.aspx?q=wsl)
    - 下载最新版本的 wsl_update_x64.msi 
    - 彻底关闭当前的 WSL，更新内核 —— 以管理员身份运行 `wsl --shutdown`；运行第2步下载的文件，完成内核更新
    - 再次开启 WSL，检测内核版本 —— 以管理员身份运行 `wsl， uname -a`
1. 在Windows系统中安装AMD显卡驱动程序(针对wsl)
    - 显卡驱动地址 [https://www.amd.com/en/support/kb/release-notes/rn-rad-win-wsl-support](https://www.amd.com/en/support/kb/release-notes/rn-rad-win-wsl-support)
1. 进入wsl子系统，安装AMD的ROCm
    - 安装方法 [https://rocmdocs.amd.com/en/latest/deploy/linux/quick_start.html](https://rocmdocs.amd.com/en/latest/deploy/linux/quick_start.html)
1. 进入wsl子系统，安装`miniconda`，并设置一个虚拟环境（`directml`）
    - 操作指南[https://learn.microsoft.com/zh-cn/windows/ai/directml/gpu-pytorch-wsl](https://learn.microsoft.com/zh-cn/windows/ai/directml/gpu-pytorch-wsl)
    - 上述网页上的Set up a Python environment这一节
1. 在虚拟环境`directml`中安装`PyTorch`
    <center><img src="../img/5_deep_learning_computation/pytorchAMD.png" width=100%></center>
1. 最后装`DirectML`
    - `pip install torch-directml`

## 计算设备



- 可以指定用于存储和计算的设备，如CPU和GPU
- 默认情况下，张量是在内存中创建的，然后使用CPU计算它


- 在`PyTorch`中，每个数组都有一个设备（device），通常将其称为环境（context）
- 默认情况下，所有变量和相关的计算都分配给CPU
- 也可以指定环境是GPU

### 设置计算环境

- 在`PyTorch`中，CPU环境用`torch.device('cpu')`表示
- GPU环境
    - 安装了NVIDIA显卡，用`torch.device('cuda')`表示
    - mac的M1系统，用`torch.device('mps')`表示
    - 安装AMD的显卡，用`torch_directml.device()`表示

- 应该注意的是，CPU设备意味着所有物理CPU和内存，这意味着`PyTorch`的计算将尝试使用所有CPU核心

- GPU设备只代表一个卡和相应的显存
- 如果有多个GPU，我们使用`torch.device(f'cuda:{i}')`来表示第$i$块GPU（$i$从0开始）
- 另外，`cuda:0`和`cuda`是等价的

### 查看系统中的计算环境

#### NVIDIA

In [2]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

- 可以查询可用gpu的数量


In [3]:
# 查询可用gpu的数量

torch.cuda.device_count()

2

- 定义了两个方便的函数，允许在不存在所需所有GPU的情况下运行代码


In [10]:
def try_gpu(i=0):
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cpu'), device(type='cpu'), [device(type='cpu')])

#### AMD

In [3]:
import torch_directml

In [2]:
torch_directml.is_available()  # GPU设备是否存在

True

In [4]:
torch_directml.device_count() # GPU设备数量

2

In [6]:
torch_directml.device_name(0)  # GPU设备名称
torch_directml.device_name(1)

'Radeon 520\x00'

'Intel(R) UHD Graphics 630\x00'

- 设置GPU设备

In [4]:
dml = torch_directml.device(0)

#### Apple

In [7]:
torch.device('mps')

device(type='mps')

## 张量与GPU

- 可以查询张量所在的设备，默认情况下，张量是在CPU上创建的

In [11]:
x = torch.tensor([1, 2, 3])

x.device  # 获取该张量的计算环境

device(type='cpu')

> 需要注意的是，无论何时要对多个项进行操作，它们都必须在**同一个设备上**

### 存储在GPU上

#### NVIDIA

- 可以在创建张量时指定存储设备
- 例如，在第一个GPU上创建张量`X`
- 在GPU上创建的张量只消耗这个GPU的显存。一般来说，需要确保不创建超过GPU显存限制的数据

In [13]:
X = torch.ones(2, 3, device=try_gpu())
X
X.device

device(type='cpu')

- 假设我们至少有两个GPU，下面的代码将在第二个GPU上创建一个随机张量

In [7]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.3821, 0.5270, 0.4919],
        [0.9391, 0.0660, 0.6468]], device='cuda:1')

#### AMD

In [5]:
X1 = torch.ones(2,3,device=dml)
X1
X1.device

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='privateuseone:0')

device(type='privateuseone', index=0)

In [16]:
# 在第二个GPU上创建一个随机张量
dml1 = torch_directml.device(1)
Y1 = torch.rand(2,3,device=dml1)
Y1

tensor([[0.4452, 0.2311, 0.8754],
        [0.8269, 0.8915, 0.6630]], device='privateuseone:1')

#### Apple

In [9]:
X2_0 = torch.ones(2,3)
X2_0
X2_0.device

tensor([[1., 1., 1.],
        [1., 1., 1.]])

device(type='cpu')

In [12]:
X2 = torch.ones(2,3,device='mps')
X2

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='mps:0')

In [11]:
Y2 = torch.rand(2,3,device='mps')
Y2

tensor([[0.6545, 0.2072, 0.0933],
        [0.5207, 0.9099, 0.9852]], device='mps:0')

### 复制

- 如果要计算`X + Y`，需要决定在哪里执行这个操作
- 由于`Y`位于第二个GPU上，可以将`X`传输到第二个GPU并在那里执行操作

<center><img src="../img/5_deep_learning_computation/copyto.svg" with=80%></center>

- **不要**简单地`X`加上`Y`，因为这会导致异常

#### NVIDIA

In [8]:
Z = X.cuda(1)  # 将张量X复制到GPU1
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:1')


In [9]:
# 现在数据在同一个GPU上（`Z`和`Y`都在），可以将它们相加

Y + Z

tensor([[1.3821, 1.5270, 1.4919],
        [1.9391, 1.0660, 1.6468]], device='cuda:1')

- 假设变量`Z`已经存在于第二个GPU上，如果还是调用`Z.cuda(1)`会发生什么？
    - 它将返回`Z`，而不会复制并分配新内存

In [10]:
Z.cuda(1) is Z

True

#### AMD

In [17]:
X1+Y1

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, privateuseone:0 and privateuseone:1!

In [18]:
Z1 = X1.to(dml1)
Z1

tensor([[0.8269, 0.8915, 0.6630],
        [0.8269, 0.8915, 0.6630]], device='privateuseone:1')

In [19]:
Z1+Y1

tensor([[1.2722, 1.1226, 1.5385],
        [1.6539, 1.7831, 1.3260]], device='privateuseone:1')

In [20]:
Z1.to(dml1) is Z1

True

#### Apple

In [13]:
X2_0+Y2

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, mps:0 and cpu!

In [10]:
Z2 = X2_0.to('mps')
Z2

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='mps:0')

In [14]:
Z2+Y2

tensor([[1.6545, 1.2072, 1.0933],
        [1.5207, 1.9099, 1.9852]], device='mps:0')

## 神经网络与GPU

### NVIDIA

- 神经网络模型可以指定设备

In [11]:
# 下面的代码将模型参数放在GPU上

net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

- 当输入为GPU上的张量时，模型将在同一GPU上计算结果

In [12]:
net(X)

tensor([[-0.0605],
        [-0.0605]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [13]:
# 确认一下模型参数存储在同一个GPU上

net[0].weight.data.device

device(type='cuda', index=0)

### AMD

In [6]:
net1 = nn.Sequential(nn.Linear(3, 1))
net1 = net1.to(dml)

In [7]:
net1(X1)

tensor([[-0.3361],
        [-0.3361]], device='privateuseone:0', grad_fn=<AddmmBackward0>)

In [8]:
net1[0].weight.data.device

device(type='privateuseone', index=0)

### Apple

In [15]:
net2 = nn.Sequential(nn.Linear(3, 1))
net2 = net2.to('mps')

In [16]:
net2(X2)

tensor([[0.3047],
        [0.3047]], device='mps:0', grad_fn=<LinearBackward0>)

In [18]:
net2[0].weight.device

device(type='mps', index=0)